# Structure gemini outputs in a robust way

In [2]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="List a few asian soups recipes, a yummy description and list the ingredients"
)
print(response.text)

Here are a few iconic Asian soup recipes, complete with yummy descriptions and key ingredients:

---

### 1. Vietnamese Pho Bo (Beef Noodle Soup)

**Yummy Description:**
Imagine a deeply aromatic, clear beef broth, simmered for hours with fragrant star anise, cinnamon, and ginger, creating a symphony of savory notes. It's poured over slippery flat rice noodles, thinly sliced rare beef (which cooks instantly in the hot broth), and tender pieces of brisket. Topped generously with fresh herbs like cilantro and basil, crunchy bean sprouts, a squeeze of lime, and a fiery slice of jalapeño, it's a bowl of pure, comforting magic – complex, light, and utterly addictive. Each spoonful is a healing embrace.

**Key Ingredients:**

*   **For the Broth:**
    *   Beef bones (marrow and knuckle bones)
    *   Beef brisket or flank steak
    *   Onions, ginger (charred)
    *   Whole spices: Star anise, cinnamon sticks, cloves, black cardamom pods, coriander seeds
    *   Fish sauce, rock sugar, salt

In [3]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="List a few asian soups recipes, a yummy description and list the ingredients. Structure it in json format"
)
print(response.text)

```json
[
  {
    "name": "Pho Bo (Vietnamese Beef Noodle Soup)",
    "description": "A fragrant, soul-warming Vietnamese beef noodle soup, celebrated for its clear, deeply savory broth simmered for hours with star anise, cinnamon, and ginger. Each slurp delivers tender slices of beef, slippery rice noodles, and a fresh burst of herbs like basil and cilantro, creating a symphony of balanced flavors that comfort and invigorate.",
    "ingredients": [
      "Beef bones (marrow and knuckle bones)",
      "Beef brisket or chuck",
      "Fish sauce",
      "Yellow rock sugar (or brown sugar)",
      "Onions",
      "Ginger",
      "Star anise",
      "Cinnamon sticks",
      "Black cardamom pods",
      "Cloves",
      "Coriander seeds",
      "Rice noodles (banh pho)",
      "Thinly sliced raw beef (sirloin, eye of round)",
      "Fresh herbs: cilantro, Thai basil, mint",
      "Bean sprouts",
      "Lime wedges",
      "Sliced chilis (e.g., jalapeño, bird's eye chili)",
      "Hoisin sauc

In [5]:
from pydantic import BaseModel
from google import genai


class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]},
)

print(response.text)

[
  {
    "recipe_name": "Tom Yum Goong",
    "description": "A classic Thai hot and sour soup bursting with aromatic herbs, tender shrimp, and a spicy, tangy broth that awakens the palate.",
    "ingredients": [
      "Shrimp",
      "Lemongrass",
      "Galangal",
      "Kaffir lime leaves",
      "Mushrooms",
      "Fish sauce",
      "Lime juice",
      "Chili paste",
      "Chili peppers",
      "Cilantro"
    ]
  },
  {
    "recipe_name": "Japanese Miso Ramen",
    "description": "A comforting and savory Japanese noodle soup featuring a rich miso-infused broth, springy ramen noodles, tender pork belly, and a medley of toppings.",
    "ingredients": [
      "Ramen noodles",
      "Pork belly (chashu)",
      "Miso paste",
      "Chicken or pork broth",
      "Soy sauce",
      "Mirin",
      "Garlic",
      "Ginger",
      "Soft-boiled egg",
      "Nori",
      "Scallions",
      "Bean sprouts"
    ]
  },
  {
    "recipe_name": "Vietnamese Pho Bo",
    "description": "A beloved Vi

In [6]:

recipes = response.parsed
type(recipes), len(recipes)


(list, 3)

In [7]:
recipes[0]

Recipe(recipe_name='Tom Yum Goong', description='A classic Thai hot and sour soup bursting with aromatic herbs, tender shrimp, and a spicy, tangy broth that awakens the palate.', ingredients=['Shrimp', 'Lemongrass', 'Galangal', 'Kaffir lime leaves', 'Mushrooms', 'Fish sauce', 'Lime juice', 'Chili paste', 'Chili peppers', 'Cilantro'])

In [8]:
recipes[-1].recipe_name, recipes[-1].ingredients

('Vietnamese Pho Bo',
 ['Beef bones',
  'Beef brisket',
  'Rice noodles (banh pho)',
  'Onions',
  'Ginger',
  'Star anise',
  'Cinnamon stick',
  'Cardamom pods',
  'Fish sauce',
  'Sugar',
  'Lime',
  'Bean sprouts',
  'Thai basil',
  'Cilantro',
  'Jalapenos'])

# Simulate house prices

In [12]:
from typing import Literal


class Home(BaseModel):
    price: int
    monthly_fee: int
    living_area: float
    number_rooms: float
    address: str
    type: Literal["apartment", "house"]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""
    List 50 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK.
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Home]},
)

homes= response.parsed
homes[:3]


[Home(price=3800000, monthly_fee=3500, living_area=75.5, number_rooms=3.0, address='Vasagatan 23B, Stockholm', type='apartment'),
 Home(price=6200000, monthly_fee=1200, living_area=145.0, number_rooms=5.0, address='Björkvägen 12, Nacka', type='house'),
 Home(price=2750000, monthly_fee=4200, living_area=60.0, number_rooms=2.5, address='Storgatan 15, Göteborg', type='apartment')]

In [13]:
import pandas as pd

df = pd.DataFrame([dict(home) for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,address,type
0,3800000,3500,75.5,3.0,"Vasagatan 23B, Stockholm",apartment
1,6200000,1200,145.0,5.0,"Björkvägen 12, Nacka",house
2,2750000,4200,60.0,2.5,"Storgatan 15, Göteborg",apartment
3,4900000,850,110.0,4.0,"Ekebergsvägen 7, Lund",house
4,1950000,2800,40.0,1.5,"Södra Förstadsgatan 88, Malmö",apartment


In [14]:
df.query("price < 2_000_000")

,price,monthly_fee,living_area,number_rooms,address,type
4,1950000,2800,40.0,1.5,"Södra Förstadsgatan 88, Malmö",apartment
12,1700000,2500,35.0,1.0,"Järnvägsgatan 12, Helsingborg",apartment
34,1800000,2600,38.0,1.5,"Stortorget 5, Ystad",apartment
